In [ ]:
import sys
import os

# プロジェクトのルートディレクトリを指定
project_root = os.path.abspath(os.path.join(os.getcwd(), './..'))
sys.path.append(project_root)

# current_dir = os.getcwd()
# os.environ['PROJECT_ROOT'] = os.path.join(current_dir, '..')

In [ ]:
from omegaconf import OmegaConf
from config.modifier import dynamically_modify_train_config
config_paths = [
        '../config/dataset/gen1/event_frame/single/base.yaml',
        '../config/model/rvt_detector/rvt_frame.yaml',
        '../config/experiment/single/train.yaml',
    ]

configs = [OmegaConf.load(path) for path in config_paths]
merged_conf = OmegaConf.merge(*configs)
dynamically_modify_train_config(config=merged_conf)

In [ ]:
from modules.fetch import fetch_data_module, fetch_model_module
import torch

data = fetch_data_module(merged_conf)
data.setup('fit')
model = fetch_model_module(merged_conf)
model.setup('fit')

ckpt_path = '../scripts/result/gen1/rvt-t/event_frame-dt50/20241116-134504/train/epoch=49-val_AP=0.42.ckpt'
ckpt = torch.load(ckpt_path, map_location=torch.device('cpu'))
model.load_state_dict(ckpt['state_dict'])

In [ ]:
from utils.visualize import save_sequence_with_pred
rnn_model = model.model
rnn_model.eval()
save_sequence_with_pred(dataloader=data.val_dataloader(),
                        t_ms=100,
                        output_file='test.mp4',
                        model=rnn_model,
                        type='gen1',
                        mode='val',)
